In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score, precision_score, confusion_matrix, accuracy_score

In [3]:
df = pd.read_csv("evgeni_set.csv")
df['age'] = df['age']/110 #нормализация

#Меня не устраивает что  в train_test_split может в тестовую выборку попасть 0 инфарктов или все 100
#поэтому разделю вручную и соеденю после деления обратно
df_heartattack_x = df[df['minf']==1].iloc[:, 1:]
df_heartattack_y = df[df['minf']==1].iloc[:, 0]
df_no_heartattack_x = df[df['minf']==0].iloc[:, 1:]
df_no_heartattack_y = df[df['minf']==0].iloc[:, 0]

df_heartattack_x_train, df_heartattack_x_test, df_heartattack_y_train, df_heartattack_y_test = train_test_split(df_heartattack_x, df_heartattack_y, test_size=0.2)
df_no_heartattack_x_train, df_no_heartattack_x_test, df_no_heartattack_y_train, df_no_heartattack_y_test = train_test_split(df_no_heartattack_x, df_no_heartattack_y, test_size=0.2)

x_train = pd.concat((df_heartattack_x_train, df_no_heartattack_x_train), axis=0)
y_train = pd.concat((df_heartattack_y_train, df_no_heartattack_y_train), axis=0)
x_test = pd.concat((df_heartattack_x_test, df_no_heartattack_x_test), axis=0)
y_test = pd.concat((df_heartattack_y_test, df_no_heartattack_y_test), axis=0)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
matrix = confusion_matrix(y_test, y_pred)
print(matrix)
print("recall: "+str(recall_score(y_test, y_pred)))
print("precision: "+str(precision_score(y_test, y_pred)))
print("accuracy_score: "+str(accuracy_score(y_test, y_pred)))
print("Доля обектов отнесеных к классу 1 от тестовой выборки: "+str(round((sum(matrix[:, 1])/x_test.shape[0])*100))+'%')

[[757  32]
 [  0  21]]
recall: 1.0
precision: 0.39622641509433965
accuracy_score: 0.9604938271604938
Доля обектов отнесеных к классу 1 от тестовой выборки: 7%


In [26]:
y_train_pred = model.predict(x_train)
confusion_matrix(y_train, y_train_pred)
x_train_for_pipeline = x_train[y_train_pred==1]
y_train_for_pipeline = y_train[y_train_pred==1]
x_test_for_pipeline = x_test[y_pred==1]
y_test_for_pipeline = y_test[y_pred==1]

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver="liblinear")
lr.fit(x_train_for_pipeline, y_train_for_pipeline)
y_pred_from_pipeline = lr.predict(x_test_for_pipeline)
confusion_matrix(y_test_for_pipeline, y_pred_from_pipeline)

array([[31,  1],
       [10, 11]], dtype=int64)

In [43]:
prediction = lr.predict_proba(x_test_for_pipeline)
prediction_train = lr.predict_proba(x_train_for_pipeline)
results = {'threshold':[], 'test_recall_score':[], 'test_f1_score':[], 'test_precision_score':[]
           , 'train_recall_score':[], 'train_f1_score':[], 'train_precision_score':[]}
iters = prediction[:, 1].copy()
iters.sort()

for i in iters:
    predict_mine = []
    for i2 in prediction:
        if i2[1]>i:
            predict_mine.append(1)
        else:
            predict_mine.append(0)

    train_predict_mine = []
    for i2 in prediction_train:
        if i2[1]>i:
            train_predict_mine.append(1)
        else:
            train_predict_mine.append(0)
            
    results['threshold'].append(i)
    results['test_recall_score'].append(recall_score(y_test_for_pipeline, predict_mine))
    results['test_f1_score'].append(f1_score(y_test_for_pipeline, predict_mine))
    results['test_precision_score'].append(precision_score(y_test_for_pipeline, predict_mine))
    results['train_recall_score'].append(recall_score(y_train_for_pipeline, train_predict_mine))
    results['train_f1_score'].append(f1_score(y_train_for_pipeline, train_predict_mine))
    results['train_precision_score'].append(precision_score(y_train_for_pipeline, train_predict_mine))

e:\Anaconda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\Anaconda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\Anaconda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\Anaconda\envs\machinelearning\lib\site-packages\sklearn\metrics\_classification.py:1344: Un

In [45]:
df_results = pd.DataFrame(results)
df_results

,threshold,test_recall_score,test_f1_score,test_precision_score,train_recall_score,train_f1_score,train_precision_score
0,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
1,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
2,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
3,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
4,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
5,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
6,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
7,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
8,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
9,0.166566,0.857143,0.562500,0.418605,0.952381,0.603774,0.441989
